# Обучение EasyOCR

In [ ]:
import os
from tqdm import tqdm
import shutil
import wget
import yaml
import wget
import subprocess

### Обучение

Клонируем библиотеку и копируем датасет в нужные директории

In [ ]:
if 'EasyOCR' not in os.listdir():
  subprocess.run(['git', 'clone', 'https://github.com/something-original/EasyOCR'])
  shutil.move('./train', './EasyOCR/trainer/all_data', copy_function=shutil.copy2)
  shutil.move('./val', './EasyOCR/trainer/all_data', copy_function=shutil.copy2)

Загружаем предобученный чекпоинт

In [ ]:
if 'cyrillic_g2.pth' not in os.listdir():
  wget.download('https://huggingface.co/smthrgnl/easy_ocr/blob/main/cyrillic_g2.pth')
if 'model' not in os.listdir('EasyOCR'):
  os.makedirs('EasyOCR/model')
  shutil.move('./cyrillic_g2.pth', 'EasyOCR/model')

Настраиваем параметры

In [ ]:
symbols = '.,?!:;/%-№()«»\$_@' + '\'' + '\"'

In [ ]:
params = {
  'number': '0123456789',
  'symbol': symbols,
  'lang_char': 'ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyzАБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯабвгдеёжзийклмнопрстуфхцчшщъыьэюя',
  'experiment_name': 'fmd',
  'train_data': 'all_data/train',
  'valid_data': 'all_data/val',
  'manualSeed': 1111,
  'workers': 6,
  'batch_size': 64, #32
  'num_iter': 30000,
  'valInterval': 200,
  'saved_model': 'cyrillic_g2',
  'FT': True,
  'optim': False, # значение по умолчанию - Adadelta
  'lr': 1.,
  'beta1': 0.9,
  'rho': 0.95,
  'eps': 0.00000001,
  'grad_clip': 5,

  'select_data': 'train', # это папка dataset в train_data
  'batch_ratio': '1',
  'total_data_usage_ratio': 1.0,
  'batch_max_length': 68,
  'imgH': 64,
  'imgW': 600,
  'rgb': False,
  'contrast_adjust': False,
  'sensitive': True,
  'PAD': True,
  'contrast_adjust': 0.0,
  'data_filtering_off': False,

  'Transformation': 'None',
  'FeatureExtraction': 'VGG',
  'SequenceModeling': 'BiLSTM',
  'Prediction': 'CTC',
  'num_fiducial': 20,
  'input_channel': 1,
  'output_channel': 256,
  'hidden_size': 256,
  'decode': 'greedy',
  'new_prediction': False,
  'freeze_FeatureFxtraction': False,
  'freeze_SequenceModeling': False
}

In [ ]:
with open('EasyOCR/trainer/config_files/custom_data_train.yaml', 'w') as file:
    yaml.dump(params, file)

#### Запуск обучения

Устанавливаем зависимости фреймворка

In [ ]:
!pip install -r EasyOCR/requirements.txt

In [ ]:
import sys

sys.path.insert(0, './EasyOCR/trainer')

In [ ]:
import os
import torch.backends.cudnn as cudnn
import yaml
from train import train
from utils import AttrDict
import pandas as pd

cudnn.benchmark = True
cudnn.deterministic = False

def get_config(file_path):
    with open(file_path, 'r', encoding="utf8") as stream:
        opt = yaml.safe_load(stream)
    opt = AttrDict(opt)
    if opt.lang_char == 'None':
        characters = ''
        for data in opt['select_data'].split('-'):
            csv_path = os.path.join(opt['train_data'], data, 'labels.csv')
            df = pd.read_csv(csv_path, sep='^([^,]+),', engine='python', usecols=['filename', 'words'], keep_default_na=False)
            all_char = ''.join(df['words'])
            characters += ''.join(set(all_char))
        characters = sorted(set(characters))
        opt.character= ''.join(characters)
    else:
        opt.character = opt.number + opt.symbol + opt.lang_char
    os.makedirs(f'./saved_models/{opt.experiment_name}', exist_ok=True)
    return opt


In [ ]:
#Запускаем обучение
opt = get_config("EasyOCR/trainer/config_files/custom_data_train.yaml")
train(opt, amp=False)

Filtering the images containing characters which are not in opt.character
Filtering the images whose label is longer than opt.batch_max_length
--------------------------------------------------------------------------------
dataset_root: all_data/train
opt.select_data: ['train']
opt.batch_ratio: ['1']
--------------------------------------------------------------------------------
dataset_root:    all_data/train	 dataset: train


AssertionError: datasets should not be an empty iterable